In [1]:
import mimetypes
import os

import boto3
from botocore.exceptions import ClientError


def upload_file_to_s3(local_path: str, bucket: str, prefix: str) -> str:
    """
    Upload a local file to S3 under s3://{bucket}/{prefix}/<basename>.
    Returns the S3 URI.
    """
    if not os.path.isfile(local_path):
        raise FileNotFoundError(f"Local file not found: {local_path}")

    s3 = boto3.client("s3")

    filename = os.path.basename(local_path)
    prefix = prefix.strip("/")
    key = f"{prefix}/{filename}" if prefix else filename

    content_type, _ = mimetypes.guess_type(local_path)
    extra_args = {}
    if content_type:
        extra_args["ContentType"] = content_type

    try:
        s3.upload_file(local_path, bucket, key, ExtraArgs=extra_args or None)
    except ClientError as e:
        raise RuntimeError(f"S3 upload failed: {e}") from e

    return f"s3://{bucket}/{key}"

In [2]:
local_file = "titanic-competition-step-by-step-using-xgboost.ipynb"
bucket_name = "retail-mlops-edu-202602"
s3_prefix = "output"

s3_uri = upload_file_to_s3(local_file, bucket_name, s3_prefix)
print(f"Uploaded to: {s3_uri}")

Uploaded to: s3://retail-mlops-edu-202602/output/titanic-competition-step-by-step-using-xgboost.ipynb
